In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Input,Conv2DTranspose,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from PIL import Image
from PIL import UnidentifiedImageError
from skimage.metrics import peak_signal_noise_ratio as psnr,mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

def create_patch(image,patch_size):
    patches=[]
    h,w,_=image.shape
    patch_h,patch_w=patch_size
    for i in range(0,h,patch_h):
        for j in range(0,w,patch_w):
            patch=image[i:i+patch_h,j:j+patch_w]
            patches.append(patch)
    return patches

def load_images(image_dir,image_size=(1024,1024),patch_size=(256,256)):
    images=[]
    for img_name in sorted(os.listdir(image_dir)):
        img_path=os.path.join(image_dir,img_name)
        img=load_img(img_path,target_size=image_size)
        img=img_to_array(img)/255
        patches=create_patch(img,patch_size)
        images.extend(patches)
    
    return np.array(images)

def build_denoising_model(input_shape=(256,256,3)):
    tf.keras.backend.clear_session()
    input=Input(shape=input_shape)
    
    x=Conv2D(32,(3,3),activation="relu",kernel_initializer='he_normal',padding="same")(input)
    x=MaxPooling2D((2,2),padding="same")(x)
    x=Conv2D(64,(3,3),activation="relu",kernel_initializer='he_normal',padding="same")(x)
    x=MaxPooling2D((2,2),padding="same")(x)
    x=Conv2D(128,(3,3),activation="relu",kernel_initializer='he_normal',padding="same")(x)
    x=MaxPooling2D((2,2),padding="same")(x)
    
    x=Conv2DTranspose(128,(3,3),strides=2,activation="relu",kernel_initializer='he_normal',padding="same")(x)
    x=Conv2DTranspose(64,(3,3),strides=2,activation="relu",kernel_initializer='he_normal',padding="same")(x)
    x=Conv2DTranspose(32,(3,3),strides=2,activation="relu",kernel_initializer='he_normal',padding="same")(x)
    x=Conv2D(3,(3,3),activation="sigmoid",kernel_initializer='he_normal',padding="same")(x)
    
    autoencoder=Model(input,x)
    autoencoder.compile(optimizer=Adam(0.001),loss='mean_squared_error')
    return autoencoder

def train_model():
    clean_images=load_images('./Train/high')
    noisy_images=load_images('./Train/low')

    model=build_denoising_model(input_shape=(256,256,3))

    early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
    model.fit(noisy_images,clean_images,epochs=20,batch_size=32,validation_split=0.1,callbacks=[early_stopping])
    return model

def evaluate_model(model,clean_images,noisy_images):
    predictions=model.predict(noisy_images)
    mse_scores=[mse(clean,pred) for clean,pred in zip(clean_images,predictions)]
    psnr_scores=[psnr(clean,pred) for clean,pred in zip(clean_images,predictions)]
    mae_scores=[mae(clean.flatten(),pred.flatten()) for clean,pred in zip(clean_images,predictions)]
    
    print(f"Mean MSE: {np.mean(mse_scores)}")
    print(f"Mean PSNR: {np.mean(psnr_scores)}")
    print(f"Mean MAE: {np.mean(mae_scores)}")

if __name__=='__main__':
    model=train_model()
    
    clean_images=load_images('./Train/high')
    noisy_images=load_images('./Train/low')
    
    evaluate_model(model,clean_images,noisy_images)

    test_noisy_images=load_images('./test/low')
    predictions=model.predict(test_noisy_images)

    if not os.path.exists('./test/predicted'):
        os.makedirs('./test/predicted')

    for i,pred in enumerate(predictions):
        img=Image.fromarray((pred*255).astype(np.uint8))
        img.save(f'./test/predicted/pred_{i}.png')



Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 417s 2s/step - loss: 0.0371 - val_loss: 0.0255
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 393s 2s/step - loss: 0.0251 - val_loss: 0.0215
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 381s 2s/step - loss: 0.0234 - val_loss: 0.0208
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - loss: 0.0226 - val_loss: 0.0209
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 399s 2s/step - loss: 0.0223 - val_loss: 0.0217
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 504s 2s/step - loss: 0.0217 - val_loss: 0.0201
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 376s 2s/step - loss: 0.0216 - val_loss: 0.0237
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - loss: 0.0212 - val_loss: 0.0211
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 373s 2s/step - loss: 0.0193 - val_loss: 0.0211
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 355s 2s/step - loss: 0.0202 - val_loss: 0.0203
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 352s 2s/step - loss: 0.0200 - val_loss: 0.0213
Epoch 12/20
219/219 ━━━━━━━━━━━━━━━━━━━━